### Praktikum 04 - Informed Search
Nama : Arif Syafarian <br>
Kelas : 3A <br>
NPM : 243307005 <br>
***

1. Jelaskan langkah-langkah algoritma:<br>
a. Greedy Best-First Search (GBFS)<br>
b. A* Search

A. GBFS bekerja dengan cara dengan langkah sepagai berikut
    1. Inisiasi, siapkan wadah simpul hidup dan jalur
    berbentuk larik
    2. Masukkan simpul permulaan kedalam larik simpul
    hidup
    3. Keluarkan simpul dari larik simpul hidup yang
    memiliki nilai harga, f(n), yang paling kecil. Simpul
    ini menjadi simpul ekspan. Masukan simpul ini
    kedalam jalur.
    4. Jelajahi simpul ekspan, dapat kan simpul-simpul yang
    terhubung dengan simpul ekspan.
    5. Cek untuk setiap simpul yang terhubung dengan
    simpul ekspan apakah merupakan simpul tujuan, jika
    benar, masukkan simpul tujuan tersebut kedalan jalur
    kemudian hentikan iterasi. Jika bukan, periksa apakah
    simul tersebut sudah ada dalam larik simpul hidup
    atau larikk simpul ekspan, jika balum masukan simpul
    kedalam larik simpul hidup.
    6. (opsional) Evaluasi kembali nilai heuristik untuk
    setiap simpul dalam larik simpul hidup.
    7. Ulangi dari poin ke 2

2. Greedy Best-first Search Algorithm<br>
Diketahui sebuah maze/labirin pada file maze2.txt, dimana A merupakan asal dan B merupakan tujuan yang akan dicapai. Implementasikan algoritma GBFS untuk mencari rute terdekat dari A ke B.

In [8]:
import sys
import operator


class Node:
    def __init__(self, state, parent, action, heuristic):
        self.state = state
        self.parent = parent
        self.action = action
        # nilai heuristic
        self.heuristic = heuristic


class Greedy:
    def __init__(self):
        self.frontier = []

    def add(self, node):
        # print("Heuristic node baru : " + str(node.heuristic))
        # try:
        #     print("Heuristic node pertama : " + str(self.frontier[0].heuristic))
        # except IndexError:
        #     print("tidak ada node di frontier")
        self.frontier.append(node)
        # fungsi mengurutkan node di frontier berdasarkan nilai heuristic
        self.frontier.sort(key=operator.attrgetter("heuristic"))
        # print(
        #     "Huristic node pertama di frontier setelah ditambah : "
        #     + str(self.frontier[0].heuristic)
        # )

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node


class Maze:
    def __init__(self, filename):
        # Read file and set height and width of maze
        filename = "maze2.txt"
        with open(filename) as f:
            contents = f.read()

        # Validate start and goal
        if contents.count("A") != 1:
            raise Exception("maze must have exactly one start point")
        if contents.count("B") != 1:
            raise Exception("maze must have exactly one goal")

        # Determine height and width of maze
        contents = contents.splitlines()
        self.height = len(contents)
        self.width = max(len(line) for line in contents)

        # Keep track of walls
        self.walls = []
        for i in range(self.height):
            row = []
            for j in range(self.width):
                try:
                    if contents[i][j] == "A":
                        self.start = (i, j)
                        row.append(False)
                    elif contents[i][j] == "B":
                        self.goal = (i, j)
                        row.append(False)
                    elif contents[i][j] == " ":
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                    row.append(False)
            self.walls.append(row)

        self.solution = None

    def print(self):
        solution = self.solution[1] if self.solution is not None else None
        print()
        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):
                if col:
                    print("█", end="")
                elif (i, j) == self.start:
                    print("A", end="")
                elif (i, j) == self.goal:
                    print("B", end="")
                elif solution is not None and (i, j) in solution:
                    print("*", end="")
                else:
                    print(" ", end="")
            print()
        print()

    def neighbors(self, state):
        row, col = state
        candidates = [
            ("up", (row - 1, col)),
            ("down", (row + 1, col)),
            ("left", (row, col - 1)),
            ("right", (row, col + 1)),
        ]
        result = []
        for action, (r, c) in candidates:
            if (
                0 <= r < self.height
                and 0 <= c < self.width
                and not self.walls[r][c]
            ):
                result.append((action, (r, c)))
        return result

    def solve(self):
        """Finds a solution to maze, if one exists."""

        # Keep track of number of states explored
        self.num_explored = 0

        # Initialize frontier to just the starting position
        start = Node(state=self.start, parent=None, action=None, heuristic=999)
        frontier = Greedy()
        frontier.add(start)

        # Initialize an empty explored set
        self.explored = set()

        # Keep looping until solution found
        while True:
            # If nothing left in frontier, then no path
            if frontier.empty():
                raise Exception("no solution")

            # Choose a node from the frontier
            node = frontier.remove()
            self.num_explored += 1

            # If node is the goal, then we have a solution
            if node.state == self.goal:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return

            # Mark node as explored
            self.explored.add(node.state)

            # Add neighbors to frontier
            for action, state in self.neighbors(node.state):
                if not frontier.contains_state(state) and state not in self.explored:
                    heuristic = 0
                    if self.goal[0] > state[0]:
                        heuristic = self.goal[0] - state[0]
                    else:
                        heuristic = state[0] - self.goal[0]

                    if self.goal[1] > state[1]:
                        heuristic += self.goal[1] - state[1]
                    else:
                        heuristic += state[1] - self.goal[1]

                    # print(heuristic)
                    child = Node(
                        state=state, parent=node, action=action, heuristic=heuristic
                    )
                    frontier.add(child)

    def output_image(self, filename, show_solution=True, show_explored=False):
        from PIL import Image, ImageDraw

        cell_size = 50
        cell_border = 2

        # Create a blank canvas
        img = Image.new(
            "RGBA", (self.width * cell_size, self.height * cell_size), "black"
        )
        draw = ImageDraw.Draw(img)
        solution = self.solution[1] if self.solution is not None else None

        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):
                # Walls
                if col:
                    fill = (40, 40, 40)
                # Start
                elif (i, j) == self.start:
                    fill = (255, 0, 0)
                # Goal
                elif (i, j) == self.goal:
                    fill = (0, 171, 28)
                # Solution
                elif solution is not None and show_solution and (i, j) in solution:
                    fill = (220, 235, 113)
                # Explored
                elif solution is not None and show_explored and (i, j) in self.explored:
                    fill = (212, 97, 85)
                # Empty cell
                else:
                    fill = (237, 240, 252)

                # Draw cell
                draw.rectangle(
                    (
                        [
                            (j * cell_size + cell_border, i * cell_size + cell_border),
                            (
                                (j + 1) * cell_size - cell_border,
                                (i + 1) * cell_size - cell_border,
                            ),
                        ]
                    ),
                    fill=fill,
                )

        img.save(filename)
        
if len(sys.argv) != 2:
    sys.exit("Usage in terminal: python maze.py maze.txt")

m = Maze(sys.argv[1])

print("Maze:")
m.print()
print("Solving...")
m.solve()
print("States Explored:", m.num_explored)
print("Solution:")
m.print()
m.output_image("result/maze2.png", show_explored=True)

Maze:

███                 █████████
█   ███████████████████   █ █
█ ████                █ █ █ █
█ ███████████████████ █ █ █ █
█                     █ █ █ █
█████████████████████ █ █ █ █
█   ██                █ █ █ █
█ █ ██ ███ ██ █████████ █ █ █
█ █    █   ██B█         █ █ █
█ █ ██ ████████████████ █ █ █
███ ██             ████ █ █ █
███ ██████████████ ██ █ █ █ █
███             ██    █ █ █ █
██████ ████████ ███████ █ █ █
██████ ████             █   █
A      ██████████████████████

Solving...
States Explored: 54
Solution:

███                 █████████
█   ███████████████████   █ █
█ ████                █ █ █ █
█ ███████████████████ █ █ █ █
█                     █ █ █ █
█████████████████████ █ █ █ █
█   ██********        █ █ █ █
█ █ ██*███ ██*█████████ █ █ █
█ █****█   ██B█         █ █ █
█ █*██ ████████████████ █ █ █
███*██             ████ █ █ █
███*██████████████ ██ █ █ █ █
███****         ██    █ █ █ █
██████*████████ ███████ █ █ █
██████*████             █   █
A******█████████████

3. Greedy Best-First-Search Algorithm (2)<br>
Program menghitung jarak terpendek antara dua kota dari file yang berisi daftar kota menggunakan algoritma GBFS.

In [ ]:
import heapq

# Inisialisasi dictionary kosong untuk mepresentasikan graf jaringan jalan di Romania.
jalur = {}

# Definisi kelas PriorityQueue untuk implementasi antrian prioritas.
class PriorityQueue:
    def __init__(self) -> None:
        self.cities = [] # inisialisasi daftar kosong utnuk menyimpan elemen antrian prioritas.
        
    def isEmpty(self):
        if self.cities == []:
            return True
        return False
    
    def check(self):
        print(self.cities)

    def push(self, city, cost):
        heapq.heappush(self.cities, (cost, city)) # Menambahkan elemen ke antrian prioritas berdasarkan biaya.

    def pop(self):
        return heapq.heappop(self.cities)[1] # Menghapus dan mengembalikan elemen dengan biaya terendah
    
# Definisi kelas CityNode untuk merepresentasikan node kota dalam graf.
class CityNode:
    def __init__(self, city, distance) -> None:
        self.city = str(city)
        self.distance = str(distance)

def MakeDict():
    file_jalan = "greedy/jalan.txt" # Ganti sesuai path masing masing"
    file = open(file_jalan, 'r')
    for str in file:
        delimeter = str.split(',')
        city1 = delimeter[0]
        city2 = delimeter[1]
        dist = delimeter[2]

        jalur.setdefault(city1, []).append(CityNode(city2, dist))

# Fungsi untuk membaca file teks yang berisi nilai heuristik dan membangun dictyionary nilai heuristik "h"
def makeHeuristicDict():
    h = {}
    file_heuristic_jalan = "greedy/heuristic_jalan.txt" #ganti path sesuai device
    file = open(file_heuristic_jalan, 'r')
    for str in file:
        delimeter = str.strip().split(',')
        node = delimeter[0].strip()
        sld = int(delimeter[1].strip()) # Jalan lurus
        h[node] = sld

    return h

# Fungsi heuristik yang digunakan dalam pencarian GBFS.
def heuristic(node, values):
    return values[node]

# Fungsi utama yang menjalankan algoritma GBFS.
def greedyBFS(start, end):
    path = {}
    q = PriorityQueue()
    h = makeHeuristicDict()

    q.push(start, 0)
    path[start] = None
    expeand_list = []

    while q.isEmpty() == False:
        curr = q.pop()
        expeand_list.append(curr)

        if curr == end:
            break

        for new in jalur[curr]:
            if new.city not in path:
                f_cost = heuristic(new.city, h)
                q.push(new.city, f_cost)
                path[new.city] = curr

    # Fungsi untuk mencetak output hasil pencarian.
    printOutput(start, end, path, expeand_list)

def printOutput(start, end, path, expandlist):
    finalpath = []
    i = end

    while(path.get(i) != None):
        finalpath.append(i)
        i = path[i]
    finalpath.append(start)
    finalpath.reverse()

    print("Program Greedy Best-First Search")
    print(start + " => " + end)
    print("Kota yang dilewati dengan jarak terpendek\t: " + str(finalpath))

# Fungsi utama yang dijalankan saat skrip ini dieksekusi.

def main():
    src = "Arad"
    dst = "Bucharest"
    MakeDict()
    greedyBFS(src, dst)

# Pengecekan apakah skrip ini dijalankan sebagai program utama.
if __name__ == "__main__":
    main()

Program Greedy Best-First Search
Arad=> Bucharest
Kota yang dilewati dengan jarak terpendek	: ['Arad', 'Sibiu', 'Fagaras', 'Bucharest']


4. Implementasi Algoritma A* Search.

5. Studi  Kasus:  Implementasikan  algortima  Greedy  Best  First  Search  untuk  mencari rute dari A menuju Z pada suatu labirin.

6. Studi Kasus: Diketahui sebuah graph yang berisi node-node S, A, B, C, D, E, G pada soal.<br>
a. Buatlah struktur pohonnya yang dimulai dari node S.<br>
b. Buatlah sebuah program python untuk menemukan jalur dari S ke G berdasarkan gambar di bawah ini menggunakan Algoritma A*.

7. Studi Kasus: Buatlah sebuah program Python untuk mencari rute terbaik berdasarkan pada list kota di Romania sesuai soal nomor 3 di atas, yang memungkinkan user untuk menggunakan pilihan algoritma BFS, DFS, UCS, GBFS, dan A* berdasarkan inputan berupa kota asal dan kota tujuan.